In [193]:
import csv
import cv2
import glob
import os
import numpy as np
import xml.etree.ElementTree as ET

In [194]:
DATASET_FOLDER = "./images/"
TRAIN_OUTPUT_FILE = "./train.csv"
VALIDATION_OUTPUT_FILE = "./validation.csv"

In [195]:
SPLIT_RATIO = 0.8

In [196]:
if not os.path.exists(DATASET_FOLDER):
    print("Dataset not found")

In [197]:
class_names = {}
k = 0
output = []
xml_files = glob.glob("./annotations/xmls/"+'*.xml')

In [198]:
len(xml_files)

3686

In [200]:
for i, xml_file in enumerate(xml_files):
    tree = ET.parse(xml_file)
    
    path = os.path.join(DATASET_FOLDER, tree.findtext("filename"))
    
    
    height = int(tree.findtext("./size/height"))
    width = int(tree.findtext("./size/width"))
    xmin = int(tree.findtext("./object/bndbox/xmin"))
    ymin = int(tree.findtext("./object/bndbox/ymin"))
    xmax = int(tree.findtext("./object/bndbox/xmax"))
    ymax = int(tree.findtext("./object/bndbox/ymax"))
    
    
    basename = os.path.basename(path)
    basename = os.path.splitext(basename)[0]
    
    class_name = basename[:basename.rfind("_")].lower()
    
    if class_name not in class_names:
        class_names[class_name] = k
        k += 1
    
    output.append((path, height, width, xmin, ymin, xmax, ymax, class_name, class_names[class_name]))

In [210]:
output.sort(key= lambda tup : tup[-1])

In [268]:
lengths = []
for row in output:
    lengths.append(row[-2])

In [272]:
lengths = list(np.unique(lengths, return_counts=True)[1])

In [300]:
train_df = pd.DataFrame(columns=['path', 'height', 'width', 'xmin', 'ymin', 'xmax',' ymax', 'class_name', 'class_id'])
test_df = pd.DataFrame(columns=['path', 'height', 'width', 'xmin', 'ymin', 'xmax',' ymax', 'class_name', 'class_id'])

# Training Preprocessed data to output file

In [301]:
s = 0

for c in lengths:
    for i in range(c):

        path, height, width, xmin, ymin, xmax, ymax, class_name, class_id = output[s]

        if xmin >= xmax or ymin >= ymax or xmax > width or ymax > height or xmin < 0 or ymin < 0:
            print("Warning: {} contains invalid box. Skipped...".format(path))
            continue
            
            

        if i<= SPLIT_RATIO*c:
            train_df = train_df.append({
                'path': path, 'height': height, 'width': width, 'xmin' :xmin, 'ymin': ymin,
                'xmax': xmax,' ymax' : ymax, 'class_name': class_name, 'class_id': class_id
            }, ignore_index=True )
            
        else:
            test_df = test_df.append({
                'path': path, 'height': height, 'width': width, 'xmin' :xmin, 'ymin': ymin,
                'xmax': xmax,' ymax' : ymax, 'class_name': class_name, 'class_id': class_id
            }, ignore_index=True )
        s += 1 

print("Done...")

Done...


In [303]:
train_df.to_csv(TRAIN_OUTPUT_FILE, index=False)
test_df.to_csv(VALIDATION_OUTPUT_FILE, index=False)